This notebook and the accompanying `utils.py` file contain all the **Python code** for the **analysis and predictive modeling of electricity demand in Germany**.

The work was conducted in 3 steps as follows:

1.  **Forecasting the Electricity Load: Approach using Linear Models and Seasonal Filtering**
2.  **Forecasting the Electricity Price - Approach Using Machine Learning** 
3.  **Forecasting the Electricity Price - Comparisons and Conclusions** 

The project generated **three reports** detailing the methodology and results. for the **detailed discussion** of these three, please refer to the reports located in the **`reports` directory on GitHub**.

#### Package imports

In [ ]:
# Package import

from utils import *
import utils
from importlib import reload
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from tqdm import tqdm
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
import numpy as np
import pandas as pd
from datetime import datetime
import time
import holidays
import seaborn as sns

import warnings

warnings.filterwarnings("ignore")

#### Forecasting the Electricity Load: Approach using Linear Models and Seasonal Filtering

In [ ]:
# Load the basic data and call preprocessing function to incorporate the rest of the features.
# Uncomment below to re-run the entire preprocessing. May take some time. We can directly use the data_part1_merged dataframe below

# data = pd.read_csv("external_data/realized_power_consumption_2015_2024_hourly.csv", delimiter=";")
# df_load = clean_data_1(data.copy())

In [ ]:
# Avoid preprocessing time of the function clean_data above and we load directly the table in a csv ready file.

df_load = pd.read_csv("derived_data/data_part1_merged.csv", parse_dates=["datetime_clean"])

In [ ]:
df_load

In [ ]:
# Display the monthly smoothed load. You may need to run this cell multiple time to Obtain the same 
# template style as I did.

plt.figure(figsize=(12, 5))

df_idx_time = df_load.set_index('datetime_clean')
df_monthly = df_idx_time.resample('M').mean(numeric_only=1)
df_monthly.reset_index(inplace=True)

rolling_avg = df_monthly['total_load_clean'].rolling(window=4).mean()

# Plot the original data and the smoothed data
plt.plot(df_monthly["datetime_clean"], rolling_avg, color="Black", label='Smoothed Monthly Load')

plt.grid(True)
plt.style.use('ggplot')
plt.tight_layout()
plt.xlabel('Date', labelpad=20, fontsize=15)
plt.ylabel('Load (MW)',labelpad=20, fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend()
plt.show()

In [ ]:
# Plot the ACF.

plot_acf(df_load['total_load_clean'], lags=80, alpha=0.1, zero=False)
plt.xlabel('Lag', labelpad=11, fontsize=14)
plt.ylabel('Autocorrelation', labelpad=11, fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title('')
plt.show()

In [ ]:
# Plot the PACF.

plot_pacf(df_load['total_load_clean'], lags=80, method="ywm", alpha=0.1, zero=False)
plt.xlabel('Lag', labelpad=11, fontsize=14)
plt.ylabel('Partial autocorrelation', labelpad=11, fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title('')
plt.show()

In [ ]:
# Set our time series and measure different AIC values across individual lags.

ts_data = df_load.set_index('datetime_clean')
ts_data.index.freq = "H"

lags = range(1,50)
aic = list()
for i in lags:
    model = sm.tsa.AutoReg(ts_data['total_load_clean'], lags=i)
    res = model.fit()
    
    aic.append(res.aic)
    
    forecast = res.predict(start=i, end=len(ts_data)-1)
    actual_values = ts_data['total_load_clean'][i:]

In [ ]:
# Plot the AIC against Lags

plt.figure(figsize=(12, 4))
plt.plot(lags, aic, label='AIC', color="Black")
# threshold_value = 1.637e6  # Example threshold value
# plt.axhline(y=threshold_value, color=(1.0, 0.5, 0.5), linestyle='--', label='Threshold')

# Add titles and labels
plt.xlabel('Lags', fontsize=14, labelpad=11)
plt.ylabel('Criterion Value', fontsize=14, labelpad=11)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend()

# Show the plot
plt.show()

In [ ]:
# Choice of lag = 3 to be in pursuit.

p=3
model = sm.tsa.AutoReg(ts_data['total_load_clean'], lags=p)
res = model.fit()
forecast = res.predict(start=3, end=len(ts_data)-1)
actual_values = ts_data['total_load_clean'][3:]
residuals = actual_values - forecast

In [ ]:
# MSFE for the AR(3)

msfe = np.sqrt(((forecast - actual_values) ** 2).mean())

print(f"Mean Squared Forecast Error for the AR(3): {msfe:.2f}")

In [ ]:
# Subsetting the forecasts to include only values between 8 PM and 8 AM
# according to the function already above. It may take some moments to load!

df_res = pd.DataFrame(columns=['actual', 'forecast'])
df_res["actual"] = actual_values
df_res["forecast"] = forecast

df_res_subset = res_subset(ts_data, actual_values, forecast)

In [ ]:
# Scatter Plot of errors and actual values of the predefined interval

y_pred = df_res_subset["forecast"][37983:]
y_true = df_res_subset["actual"][37983:]

plt.figure(figsize=(14, 6))
sns.scatterplot(x=y_true, y=y_pred, color="red", alpha=0.4)
plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], color='black', linestyle='--')
plt.xlabel('Actual Values', labelpad=14, fontsize=14)
plt.ylabel('Predicted Values' , labelpad=14, fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
# MSFE of subsample previously plotted (Betweeen 8 PM and 8 AM).

msfe = np.sqrt(((y_true - y_pred) ** 2).mean())

print(f"Mean Squared Forecast Error of the sub sample: {msfe:.2f}")

In [ ]:
# Lag the variables.

df_ols_copy = df_load.copy()

for col in df_ols_copy.columns:
    if col != "datetime_clean" and col!= "total_load_clean" and col!= "holiday":
        df_ols_copy[f"{col}_lagged"] = df_ols_copy[col].shift(1)
        df_ols_copy.drop(col,inplace=True, axis=1)
        
df_ols_copy["total_load_clean_lag_1"] = df_ols_copy["total_load_clean"].shift(1)
df_ols_copy["total_load_clean_lag_2"] = df_ols_copy["total_load_clean"].shift(2)
df_ols_copy["total_load_clean_lag_3"] = df_ols_copy["total_load_clean"].shift(3)

df_ols_copy = df_ols_copy.dropna()
            

In [ ]:
# Perform linear predictive models for each predictor while keeping base AR(3) Model.

df_ols = df_ols_copy.copy()
df_ols.drop(["datetime_clean"], axis=1, inplace=True)
df_ols = df_ols.astype(float)

table_res = pd.DataFrame(columns=['lag', 'MSFE','aic'])

y = df_ols['total_load_clean']
X = df_ols.drop(["total_load_clean"], axis=1)

aic=dict()
i=0

for col in X.columns:
    if col not in ["total_load_clean_lag_1", "total_load_clean_lag_2", "total_load_clean_lag_3"]:
        X_subset = X.loc[:,[col, "total_load_clean_lag_1", "total_load_clean_lag_2", "total_load_clean_lag_3" ] ]
        X_subset = sm.add_constant(X_subset)
        model = sm.OLS(y, X_subset).fit()
        
        predictions = model.predict(X_subset)

        msfe = np.sqrt(((y - predictions ) ** 2).mean())
        aic[col] = model.aic
        
        table_res.at[i,"lag"] = col
        table_res.at[i,"MSFE"] = int(msfe)
        table_res.at[i,"aic"] = model.aic

        
        i+=1
        
        print(f"Mean Squared Forecast Error (MSFE) for Base model including column '{col}': {msfe:.2f}")
        print("--------------------------------------------------------------------------------------")

In [ ]:
specific_keys = list(table_res.sort_values(by="aic")[:7]["lag"].values)
specific_keys

In [ ]:
# Predicting Using the preselected features according to the AIC information Criterion.

final_model = sm.OLS(y, sm.add_constant(X[specific_keys +
                                          ["total_load_clean_lag_1",
                                           "total_load_clean_lag_2",
                                           "total_load_clean_lag_3"]])).fit()

predictions = final_model.predict(sm.add_constant(X[specific_keys +
                                                    ["total_load_clean_lag_1",
                                                     "total_load_clean_lag_2",
                                                     "total_load_clean_lag_3"]]))

msfe = np.sqrt(((y - predictions) ** 2).mean())
print(f"Mean Squared Forecast Error using features chosen according to AIC: {msfe:.2f}")

In [ ]:
# MSFE of all features lagged once.

X_spec = df_load.drop("datetime_clean", axis=1)

for col in X_spec.columns:
    if col!= "total_load_clean" and col!= "holiday" :
        X_spec[f"{col}_lagged"] = X_spec[col].shift(1)
        X_spec.drop(col,inplace=True, axis=1)

X_spec["tota_load_lag_1"] = X_spec["total_load_clean"].shift(1)
X_spec["tota_load_lag_2"] = X_spec["total_load_clean"].shift(2)
X_spec["tota_load_lag_3"] = X_spec["total_load_clean"].shift(3)

X_spec.dropna(inplace=True)
y_spec = X_spec["total_load_clean"]
X_spec.drop("total_load_clean", axis=1,inplace=True)


model = sm.OLS(y_spec, sm.add_constant(X_spec)).fit()
predictions = model.predict(sm.add_constant(X_spec))

msfe = np.sqrt(((y_spec - predictions) ** 2).mean())
print(f"Mean Squared Forecast Error of all the features: {msfe:.2f}")

In [ ]:
# Perform stepwise model selection using OLS estimator as a part of the Linear Regression estimator.

sfs = SFS(LinearRegression(),
          k_features=10,
          forward=True,
          cv=0)

sfs.fit(sm.add_constant(X), y)

In [ ]:
# Print the following best features.

selected_features = list(sfs.k_feature_names_)
print("Selected features:", selected_features)

In [ ]:
# New MSFE according to the forward stepwise model selection.

final_model = sm.OLS(y_spec, sm.add_constant(X[selected_features])).fit()
predictions = final_model.predict(sm.add_constant(X[selected_features]))

msfe = np.sqrt(((y - predictions) ** 2).mean())
print(f"Mean Squared Forecast Error According to the forward stepwise selection: {msfe:,.2f}")

In [ ]:
# Seasonal Decomposition: Decomposition according to best features selected by stepwise selection.

result_load = seasonal_decompose(ts_data["total_load_clean"] , model='additive', period=24*365)
deseasonalized_load = ts_data['total_load_clean'] - result_load.seasonal

result_temp = seasonal_decompose(ts_data["temp"] , model='additive', period=24*365)
deseasonalized_temp = ts_data['temp'] - result_temp.seasonal

result_solar = seasonal_decompose(ts_data["solar_forecast"] , model='additive', period=24*365)
deseasonalized_solar = ts_data['solar_forecast'] - result_solar.seasonal

result_co2 = seasonal_decompose(ts_data["co2_prices"] , model='additive', period=24*365)
deseasonalized_co2 = ts_data['co2_prices'] - result_co2.seasonal

result_trade = seasonal_decompose(ts_data["net_trade"] , model='additive', period=24*365)
deseasonalized_trade = ts_data['net_trade'] - result_trade.seasonal

result_gas = seasonal_decompose(ts_data["gas_prices"] , model='additive', period=24*365)
deseasonalized_gas = ts_data['gas_prices'] - result_gas.seasonal

result_fossil = seasonal_decompose(ts_data["fossil_output"] , model='additive', period=24*365)
deseasonalized_fossil = ts_data['fossil_output'] - result_fossil.seasonal

result_geothermal = seasonal_decompose(ts_data["geothermal_output"] , model='additive', period=24*365)
deseasonalized_geothermal = ts_data['geothermal_output'] - result_geothermal.seasonal

result_hydro = seasonal_decompose(ts_data["hydro_output"] , model='additive', period=24*365)
deseasonalized_hydro = ts_data['hydro_output'] - result_hydro.seasonal 

result_biomass = seasonal_decompose(ts_data["biomass_output"] , model='additive', period=24*365)
deseasonalized_biomass = ts_data['biomass_output'] - result_biomass.seasonal 

result_wind = seasonal_decompose(ts_data["wind_forecast"] , model='additive', period=24*365)
deseasonalized_wind = ts_data['wind_forecast'] - result_wind.seasonal

result_other = seasonal_decompose(ts_data["other_output"] , model='additive', period=24*365)
deseasonalized_other = ts_data['other_output'] - result_other.seasonal


deseasoned_data = pd.DataFrame({'deseasoned_load': deseasonalized_load, 
                                'deseasoned_co2': deseasonalized_co2,
                                'deseasoned_fossil':deseasonalized_fossil,
                                'deseasoned_geothermal': deseasonalized_geothermal,
                                'deseasoned_temp': deseasonalized_temp, 
                                'deseasoned_solar': deseasonalized_solar,
                                'deseasoned_hydro': deseasonalized_hydro,
                                'deseasoned_other': deseasonalized_other
                               })
     
for col in deseasoned_data.columns:
    if col != "deseasoned_load":
        deseasoned_data[f"{col}_lagged"] = deseasoned_data[col].shift(1)
        deseasoned_data.drop(col,inplace=True, axis=1)

deseasoned_data["deseasoned_load_lag_1"] = deseasoned_data["deseasoned_load"].shift(1)
deseasoned_data["deseasoned_load_lag_2"] = deseasoned_data["deseasoned_load"].shift(2)
deseasoned_data["deseasoned_load_lag_3"] = deseasoned_data["deseasoned_load"].shift(3)



deseasoned_data = deseasoned_data.dropna()
deseasoned_data.reset_index(inplace=True)
deseasoned_data = deseasoned_data.drop("datetime_clean", axis=1)

In [ ]:
# Fit the model with previously deseasoned data.

df_ols = deseasoned_data.copy()
df_ols = df_ols.astype(float)


y_deseasoned = df_ols['deseasoned_load']
X_deseasoned = df_ols.drop(["deseasoned_load"], axis=1)

X_cs = sm.add_constant(X_deseasoned)

model_deseason = sm.OLS(y_deseasoned, X_cs).fit()

predictions = model_deseason.predict(X_cs)

msfe = np.sqrt(((y_deseasoned - predictions) ** 2).mean())
print(f"Mean Squared Forecast Error of the deseasoned data: {msfe:.2f}")

In [ ]:
# Print the final predictions with the seasonality back on.

final_predictions = predictions + result_load.seasonal.reset_index().drop(
    "datetime_clean", axis=1)["seasonal"][3:].reset_index(drop=True)
print(final_predictions)

In [ ]:
# Subsetting the new deseasoned forecasts to include only values between 8 PM and 8 AM
# according to the function already above. It may take some moments to load!

final_predictions.index = ts_data.index[3:]
forecast = final_predictions

df_res = pd.DataFrame(columns=['actual', 'forecast'])
df_res["actual"] = actual_values
df_res["forecast"] = forecast

df_res_subset = res_subset(ts_data, actual_values, forecast)

In [ ]:
# Scatter Plot of errors and actual values of the predefined interval after deseasonalization.

y_pred = df_res_subset["forecast"][37983:]
y_true = df_res_subset["actual"][37983:]

plt.figure(figsize=(14, 6))
sns.scatterplot(x=y_true, y=y_pred, color="Green", alpha=0.4)
plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], color='black', linestyle='--')
plt.xlabel('Actual Values', labelpad=14, fontsize=14)
plt.ylabel('Predicted Values' , labelpad=14, fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

#### Forecasting the Electricity Price - Approach Using Machine Learning

In [ ]:
return_df = pd.read_csv("derived_data/data_part2_merged.csv", parse_dates=["datetime_clean"])
return_df

In [ ]:
# Display the monthly smoothed Price. You may need to run this cell multiple time to Obtain the same 
# template style as I did.

plt.figure(figsize=(12, 5))

return_df_idx = return_df.set_index('datetime_clean')
df_monthly = return_df_idx.resample('W').mean(numeric_only=1)
df_monthly.reset_index(inplace=True)

rolling_avg = df_monthly['Price'].rolling(window=4).mean()

# Plot the original data and the smoothed data
plt.plot(df_monthly["datetime_clean"], rolling_avg, color="Black", label='Smoothed Monthly Price')

plt.grid(True)
plt.style.use('ggplot')
plt.tight_layout()
plt.xlabel('Date', labelpad=20, fontsize=17)
plt.ylabel('Price (Eur)',labelpad=20, fontsize=17)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylim(-20, 550) 
plt.legend()
# plt.savefig("price_date_1.png", dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
# Display the monthly smoothed Price for another time interval.

plt.figure(figsize=(12, 5))
return_df_sub = return_df.loc[return_df["datetime_clean"] <= date_finder("2021-01-01", "00:00:00"),: ]


return_df_idx = return_df_sub.set_index('datetime_clean')
df_monthly = return_df_idx.resample('M').mean(numeric_only=1)
df_monthly.reset_index(inplace=True)

rolling_avg = df_monthly['Price'].rolling(window=3).mean()

# Plot the original data and the smoothed data
plt.plot(df_monthly["datetime_clean"], rolling_avg, color="Black", label='Smoothed Monthly Price')

plt.grid(True)
plt.style.use('ggplot')
plt.tight_layout()
plt.xlabel('Date', labelpad=20, fontsize=17)
plt.ylabel('Price (Eur)',labelpad=20, fontsize=17)
plt.axhline(y=np.mean(return_df_sub["Price"]), color=(1.0, 0.2, 0.2), linestyle='--', label='Average Price')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend()
# plt.savefig("price_date_2.png", dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
# Return plotting instead of the price.

plt.figure(figsize=(14, 3.5))
sns.boxplot(x=return_df['return'])

plt.tight_layout()
plt.xlabel('Return (%)',labelpad=20, fontsize=17)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

# plt.savefig("bx_1.png", dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
# Manual application of Winsorization fitted on the train set and applied on the test set too.

q01 = np.percentile(return_df.iloc[:56480,2], 0.5)  
q99 = np.percentile(return_df.iloc[:56480,2], 99.5) 

print(f"1st percentile of training data: {q01}")
print(f"99th percentile of training data: {q99}")

return_df['return'] = np.where(return_df['return'] > q99 , q99 , return_df['return'])
return_df['return'] = np.where(return_df['return'] < q01 , q01 , return_df['return'])

In [ ]:
# Simple decision tree regressor with only lagged returns.

return_ml = shifter(return_df.copy(), dependent_only=True)
X_train, X_test, y_train, y_test  = rolling_split(return_ml)

tree_reg = DecisionTreeRegressor(random_state=42)

tree_reg.fit(X_train, y_train)

y_pred = tree_reg.predict(X_test)

rmse = root_mean_squared_error(y_test, y_pred)
print(f"Simple decision tree regressor with only lagged returns: {rmse:.2f}")

In [ ]:
# Random forest regressor with only lagged returns.

return_ml = shifter(return_df.copy(), dependent_only=True)
X_train, X_test, y_train, y_test  = rolling_split(return_ml)

rf = RandomForestRegressor(random_state=42)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

rmse = root_mean_squared_error(y_test, y_pred)
print(f"Random forest regressor with only lagged returns: {rmse:.2f}")

In [ ]:
# Simple decision tree regressor with additional predictors.

return_ml = shifter(return_df.copy())
X_train, X_test, y_train, y_test  = rolling_split(return_ml)

tree_reg = DecisionTreeRegressor(random_state=42)

tree_reg.fit(X_train, y_train)

y_pred = tree_reg.predict(X_test)

rmse = root_mean_squared_error(y_test, y_pred)
print(f"Simple decision tree regressor with additional predictors: {rmse:.2f}")

In [ ]:
# Random forest regressor with additional predictors

return_ml = shifter(return_df.copy())
X_train, X_test, y_train, y_test  = rolling_split(return_ml)

rf = RandomForestRegressor(random_state=42)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

rmse = root_mean_squared_error(y_test, y_pred)
print(f"Random forest regressor with additional predictors: {rmse:.2f}")

In [ ]:
# Feature importance saving and display for the regression tree method.

feature_importances_tree = tree_reg.feature_importances_

importance_df_tree = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances_tree
}).sort_values(by='Importance', ascending=False)

# Feature importance saving and display for the regression tree method.
print(importance_df_tree)

In [ ]:
# Feature importance saving and display for the random forests method.

feature_importances_rf = rf.feature_importances_

importance_df_rf = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances_rf
}).sort_values(by='Importance', ascending=False)

print(importance_df_rf)

In [ ]:
# Individual variable fitting on each regression tree and random forest.

return_ml = shifter(return_df.copy())

tree_reg = DecisionTreeRegressor(random_state=42)
rf = RandomForestRegressor(random_state=42)

tree_eval = list()
rf_eval = list()

for idx in range(0,10):
    col_rf = [importance_df_rf.iloc[idx].Feature]
    col_tree = [importance_df_tree.iloc[idx].Feature]
    
    X_train, X_test, y_train, y_test  = rolling_split(return_ml)
    tree_reg.fit(X_train[col_tree] , y_train)
    y_pred = tree_reg.predict(X_test[col_tree])
    tree_eval.append(root_mean_squared_error(y_test, y_pred))
    
    X_train, X_test, y_train, y_test  = rolling_split(return_ml)
    rf.fit(X_train[col_rf] , y_train)
    y_pred = rf.predict(X_test[col_rf])
    rf_eval.append(root_mean_squared_error(y_test, y_pred))
    print(idx)


In [ ]:
# Display the RMSE for each variable according to the decreasing MDI Values for the regression tree method.

indices = range(1,len(rf_eval)+1)

plt.figure(figsize=(12, 4.5))

# Plot the original data and the smoothed data
plt.plot(importance_df_tree["Feature"].values, tree_eval, color="Black", 
         label='Regression Tree Model', marker='x')


plt.grid(True)
plt.style.use('ggplot')
plt.tight_layout()
plt.xlabel('Features', labelpad=20, fontsize=17)
plt.ylabel('RMSE',labelpad=20, fontsize=17)

plt.xticks(fontsize=14, rotation=45, ha='right')
plt.yticks(fontsize=14)
plt.legend()
# plt.savefig("rmse_unique.png", dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
# Display the RMSE for each variable according to the decreasing MDI Values for the random forests method.

indices = range(1,len(rf_eval)+1)

plt.figure(figsize=(12, 4.5))

# Plot the original data and the smoothed data
plt.plot(importance_df_rf["Feature"].values, rf_eval, color="Black", label='Random Forests Model', marker='x')


plt.grid(True)
plt.style.use('ggplot')
plt.tight_layout()
plt.xlabel('Features', labelpad=20, fontsize=17)
plt.ylabel('RMSE',labelpad=20, fontsize=17)

plt.xticks(fontsize=14, rotation=45, ha='right')
plt.yticks(fontsize=14)
plt.legend()
# plt.savefig("rmse_unique_2.png", dpi=400, bbox_inches='tight')
plt.show()

#### Forecasting the Electricity Price - Comparisons and Conclusions

In [ ]:
return_df = pd.read_csv("derived_data/data_part2_merged.csv", parse_dates=["datetime_clean"])
return_df

In [ ]:
# Manual application of Winsorization fitted only on the train set 
# (to avoid data leakage) and applied on the test set too.

split_index = 56480

q01 = np.percentile(return_df.iloc[:split_index,2], 0.5)  
q99 = np.percentile(return_df.iloc[:split_index,2], 99.5) 

print(f"1st percentile of training data: {q01}")
print(f"99th percentile of training data: {q99}")

return_df['return'] = np.where(return_df['return'] > q99 , q99 , return_df['return'])
return_df['return'] = np.where(return_df['return'] < q01 , q01 , return_df['return'])

In [ ]:
# Plot the ACF.

plt.style.use('ggplot')
plot_acf(return_df['return'], lags=120, alpha=0.1, zero=False)
plt.xlabel('Lag', labelpad=11, fontsize=14)
plt.ylabel('Autocorrelation', labelpad=11, fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.ylim(-0.2, 0.2)  # Set y-axis limits here
plt.title('')
# plt.savefig("Plots/acf.png", dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
# Plot the PACF.

plot_pacf(return_df['return'], lags=120, method="ywm", alpha=0.1, zero=False)
plt.xlabel('Lag', labelpad=11, fontsize=14)
plt.ylabel('Partial autocorrelation', labelpad=11, fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.ylim(-0.2, 0.2)  # Set y-axis limits here
plt.title('')
# plt.savefig("Plots/pacf.png", dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
# Set our time series and measure different AIC values across individual lags.

ts_data = return_df.set_index('datetime_clean')
ts_data.index.freq = "H"

lags = range(1,100)
aic = list()
for i in tqdm(lags, desc="Processing the lagged time series"):
    model = sm.tsa.AutoReg(ts_data['return'], lags=i)
    res = model.fit()
    aic.append(res.aic)

In [ ]:
# Plot the AIC against Lags

plt.figure(figsize=(12, 4))
plt.plot(lags, aic, label='AIC', color="Black")
# plt.axhline(y=threshold_value, color=(1.0, 0.5, 0.5), linestyle='--', label='Threshold')

# Add titles and labels
plt.xlabel('Lags', fontsize=14, labelpad=11)
plt.ylabel('Criterion Value', fontsize=14, labelpad=11)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend()

# plt.savefig("Plots/aic.png", dpi=400, bbox_inches='tight')

# Show the plot
plt.show()

In [ ]:
# Initiate the dataframe that will be used to pairwise compare all forecasts later.

final_df = pd.DataFrame()

In [ ]:
# Initiate the main dataframe to use in various fitting.

main_df= shifter(return_df.copy())
main_df.drop(["datetime_clean",
              "co2_prices_lag_1",
              "hydro_output_lag_1",
              "temp_lag_1" ], axis=1, inplace=True)

# Splitting data both for the autoregressive, the linear modeling an ensemble learning sections.

train_df, X_test, y_test = df_split(main_df)
X_train, X_test, y_train, y_test = df_split(main_df, ts=False)
X_train_em, X_test_em, y_train, y_test = df_split(main_df, ts=False)

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

# Univariate AR model with only lag 1 of the return.

model = sm.tsa.AutoReg(train_df[['return']], lags=1)
res = model.fit()
y_pred_ar_1 = res.predict(start=len(train_df)+1, end=len(main_df))

rmse = root_mean_squared_error(y_test, y_pred_ar_1)
print(f"RMSFE of the AR(1) without rolling window is: {rmse:.5f}")

final_df["y_pred_ar_1"] = pd.Series(y_pred_ar_1).values

In [ ]:
# Display model parameters

res.params

In [ ]:
# Scatter Plot of errors and actual values of the predefined interval


plt.figure(figsize=(15, 6))
sns.scatterplot(x=y_test, y=y_pred_ar_1, color="red", alpha=0.4)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='black', linestyle='--')
plt.xlabel('Actual Values', labelpad=14, fontsize=16)
plt.ylabel('Predicted Values' , labelpad=14, fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# plt.savefig("Plots/real_vs_predicted_ar_1.png", dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
# Display the forecasted values with the actual ones.

y_test.index = return_df.iloc[len(train_df)+3:,0].values
y_pred_ar_1.index = return_df.iloc[len(train_df)+3:,0].values

rolling_window = 672  

y_test_rolling = y_test.rolling(window=rolling_window).mean()
y_pred_rolling = y_pred_ar_1.rolling(window=rolling_window).mean()


# Plot the rolling averages
plt.figure(figsize=(15, 6))
plt.plot(y_test_rolling, label='True Values', alpha =0.55)
#plt.plot(predictions_rolling, label='Predictions (Rolling Avg)', color='lightcoral')
plt.plot(y_pred_rolling, label='Forecasted Values')
plt.title('')
plt.xlabel('Date',labelpad=14, fontsize=17)
plt.ylabel('Averaged Returns',labelpad=14, fontsize=17)
plt.legend()
plt.xlim(pd.Timestamp('2021-05-11'), pd.Timestamp('2024-04-15'))

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# plt.savefig("Plots/real_vs_predicted_averaged_ar_1.png", dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
# Running the rolling window.

from utils import *
import utils
reload(utils)

forecasts_array_ar = rolling_window(data = main_df[["return"]])

In [ ]:
# Looping through every feature of our data.

y_pred_lm = dict()

for col in main_df.columns:
    if col != 'return_lag_1' and col != "Price" and col!= "return":
        
        X_train_subset = X_train.loc[:,[col]]
        X_test_subset = X_test.loc[:,[col]]
        
        model = sm.OLS(y_train, X_train_subset).fit()
        predictions = model.predict(X_test_subset)
        y_pred_lm[col]= predictions
        final_df[f"y_pred_lm_{col}"] = pd.Series(y_pred_lm[col]).values

        msfe = root_mean_squared_error(y_test, predictions)
        
        print(f"Mean Squared Forecast Error (MSFE) for Base model including column '{col}': {msfe:.2f}")
        print("--------------------------------------------------------------------------------------")

In [ ]:
y_test.index = return_df.iloc[len(train_df)+3:,0].values

for col in y_pred_lm.keys():
    y_pred_lm[col].index = return_df.iloc[len(train_df)+3:,0].values
    
rolling_window = 672  # Define the window size
y_test_rolling = y_test.rolling(window=rolling_window).mean()

y_pred_rolling_solar = y_pred_lm["solar_forecast_lag_1"].rolling(window=rolling_window).mean()
y_pred_rolling_geo = y_pred_lm["geo_output_lag_1"].rolling(window=rolling_window).mean()
y_pred_rolling_other = y_pred_lm["other_output_lag_1"].rolling(window=rolling_window).mean()
y_pred_rolling_fossil = y_pred_lm["fossil_output_lag_1"].rolling(window=rolling_window).mean()

In [ ]:
# Display the forecasted values with the actual ones. Not required.
 

# Plot the rolling averages
plt.figure(figsize=(15, 6))

plt.plot(y_test_rolling, label='True Values', alpha =0.55)


plt.plot(y_pred_rolling_solar, label='Solar Forecasted Values', color="green")

plt.xlabel('Date',labelpad=14, fontsize=17)
plt.ylabel('Averaged Returns',labelpad=14, fontsize=17)
plt.legend()
plt.xlim(pd.Timestamp('2021-05-11'), pd.Timestamp('2024-04-15'))

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# plt.savefig("Plots/real_vs_predicted/real_vs_predicted_averaged_solar.png", dpi=400, bbox_inches='tight')
plt.show()


In [ ]:
# Display the forecasted values with the actual ones. Not required.
 
# Plot the rolling averages
plt.figure(figsize=(15, 6))

plt.plot(y_test_rolling, label='True Values', alpha =0.4)


plt.plot(y_pred_rolling_other, label='Other output Forecasted Values', color="darkorange")

plt.xlabel('Date',labelpad=14, fontsize=17)
plt.ylabel('Averaged Returns',labelpad=14, fontsize=17)
plt.legend()
plt.xlim(pd.Timestamp('2021-05-11'), pd.Timestamp('2024-04-15'))

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# plt.savefig("Plots/real_vs_predicted/real_vs_predicted_averaged_other.png", dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
# Display the forecasted values with the actual ones. Not required.
 
# Plot the rolling averages
plt.figure(figsize=(15, 6))

plt.plot(y_test_rolling, label='True Values', alpha =0.55)

plt.plot(y_pred_rolling_geo, label='Geo Output Forcasted Valued', color= "blue")


plt.xlabel('Date', labelpad=14, fontsize=17)
plt.ylabel('Averaged Returns', labelpad=14, fontsize=17)
plt.legend()
plt.xlim(pd.Timestamp('2021-05-11'), pd.Timestamp('2024-04-15'))

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# plt.savefig("Plots/real_vs_predicted/real_vs_predicted_averaged_geo.png", dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
# Display the forecasted values with the actual ones. Not required.
 
# Plot the rolling averages
plt.figure(figsize=(15, 6))

plt.plot(y_test_rolling, label='True Values', alpha =0.4)


plt.plot(y_pred_rolling_fossil, label='Fossil Predicted Values', color="red")


plt.xlabel('Date', labelpad=14, fontsize=17)
plt.ylabel('Averaged Returns', labelpad=14, fontsize=17)
plt.legend()
plt.xlim(pd.Timestamp('2021-05-11'), pd.Timestamp('2024-04-15'))

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# plt.savefig("Plots/real_vs_predicted/real_vs_predicted_averaged_fossil.png", dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
random_indices = np.random.choice(y_test.index, size=1500, replace=False)

In [ ]:
# Scatter Plot of errors and actual values of the predefined interval


plt.figure(figsize=(15, 6))


sns.scatterplot(x=y_pred_lm["solar_forecast_lag_1"][random_indices].index, 
                y=y_pred_lm["solar_forecast_lag_1"][random_indices], color="green", 
                alpha=0.4)



plt.xlabel('Date', labelpad=14, fontsize=16)
plt.ylabel('Predicted Return Values' , labelpad=14, fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
#plt.ylim(-100, 100)
#plt.xlim(-100, 100)

# plt.savefig("Plots/real_indexed/real_indexed_single_predictor_solar.png", dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
# Scatter Plot of errors and actual values of the predefined interval

plt.figure(figsize=(15, 6))



sns.scatterplot(x=y_pred_lm["other_output_lag_1"][random_indices].index,
                y=y_pred_lm["other_output_lag_1"][random_indices], 
                color="orange", 
                alpha=0.8)


plt.xlabel('Date', labelpad=14, fontsize=16)
plt.ylabel('Predicted Return Values' , labelpad=14, fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
#plt.ylim(-100, 100)
#plt.xlim(-100, 100)

# plt.savefig("Plots/real_indexed/real_indexed_single_predictor_other.png", dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
# Scatter Plot of errors and actual values of the predefined interval


plt.figure(figsize=(15, 6))


sns.scatterplot(x=y_pred_lm["geo_output_lag_1"][random_indices].index,
                y=y_pred_lm["geo_output_lag_1"][random_indices], 
                color="blue", 
                alpha=0.8)



plt.xlabel('Date', labelpad=14, fontsize=16)
plt.ylabel('Predicted Return Values' , labelpad=14, fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
#plt.ylim(-100, 100)
#plt.xlim(-100, 100)


# plt.savefig("Plots/real_indexed/real_indexed_single_predictor_geo.png", dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
# Scatter Plot of errors and actual values of the predefined interval


plt.figure(figsize=(15, 6))
sns.scatterplot(x =y_pred_lm["fossil_output_lag_1"][random_indices].index 
                , y=y_pred_lm["fossil_output_lag_1"][random_indices], 
                color="red", alpha=0.4)


plt.xlabel('Date', labelpad=14, fontsize=16)
plt.ylabel('Predicted Return Values' , labelpad=14, fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

#plt.ylim(-100, 100)
#plt.xlim(-100, 100)

# plt.savefig("Plots/real_indexed/real_indexed_single_predictor_fossil.png", dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
# Looping through every feature of our data again. Assumably will take some time.
from utils import *
import utils
reload(utils)

y_pred_lm_rolling = dict()

for col in main_df.columns:
    if col != 'return_lag_1' and col != "Price" and col!= "return":
        
        X_subset = main_df.loc[:,[col, "return"]]
        
        y_pred_lm_rolling[col]= rolling_window(X_subset, app="lm")

        msfe = root_mean_squared_error(y_test, y_pred_lm_rolling[col])
        
        print(f"Mean Squared Forecast Error (MSFE) for Base model including column '{col}': {msfe:.2f}")
        print("--------------------------------------------------------------------------------------")

In [ ]:
for i in y_pred_lm_rolling.keys():
    for j in range(0, len(y_pred_lm_rolling[i])):
        y_pred_lm_rolling[i][j] = y_pred_lm_rolling[i][j].values[0]

In [ ]:
# Assignement to final_df

final_df["y_pred_lm_solar_forecast_lag_1_rolling"] = pd.Series(y_pred_lm_rolling["solar_forecast_lag_1"]).values
final_df["y_pred_lm_geo_output_lag_1_rolling"] = pd.Series(y_pred_lm_rolling["geo_output_lag_1"]).values
final_df["y_pred_lm_other_output_lag_1_rolling"] = pd.Series(y_pred_lm_rolling["other_output_lag_1"]).values
final_df["y_pred_lm_fossil_output_lag_1_rolling"] = pd.Series(y_pred_lm_rolling["fossil_output_lag_1"]).values

In [ ]:
# We use now the multivariate predictors in linear models.

model = sm.OLS(y_train, X_train).fit()
y_pred_lm_multi = model.predict(X_test)
msfe = root_mean_squared_error(y_test, y_pred_lm_multi)
print(f"RMSFE of the OLS method without rolling window on all features is: {msfe:.5f}")

final_df["y_pred_lm_multi"] = y_pred_lm_multi

In [ ]:
# Display model parameters.

model.params

In [ ]:
# Running the rolling window.

from utils import *
import utils
reload(utils)

forecasts_array_lm = rolling_window(main_df, app="lm")

In [ ]:
# Changing the format of the forecast and displaying the RMSFE

y_pred_lm_multi_rolling = list()
for i in range(0,len(forecasts_array_lm)):
    y_pred_lm_multi_rolling.append(forecasts_array_lm[i].values[0])
    
y_pred_lm_multi_rolling = pd.Series(y_pred_lm_multi_rolling)

rmse = root_mean_squared_error(y_test, y_pred_lm_multi_rolling)
print(f"RMSFE of the OLS method with expanding window on all features is: {rmse:.5f}")

final_df["y_pred_lm_multi_rolling"] = pd.Series(y_pred_lm_multi_rolling).values

In [ ]:
# Simple decision tree regressor with additional predictors.

tree_reg = DecisionTreeRegressor(random_state=42)

tree_reg.fit(X_train_em, y_train)

y_pred_multi_dt = tree_reg.predict(X_test_em)

rmse = root_mean_squared_error(y_test, y_pred_multi_dt)

print(f"Simple decision tree regressor with additional predictors: {rmse:.2f}")

final_df["y_pred_dt_multi"] = pd.Series(y_pred_multi_dt).values

In [ ]:
# Tracking the runtime to optimize the n_estimators parameter.

n_estimator_time_track_df = pd.DataFrame(columns=[ "n_estimators" , 'time', "RMSE"])

In [ ]:
# Random forest regressor with additional predictors

for n in tqdm(range(100, 0, -10), desc="Loading random forest: "):

    start_time = time.time()

    rf = RandomForestRegressor(n_estimators = n, random_state=42, min_samples_split= 2)

    rf.fit(X_train_em, y_train)

    y_pred_multi_rf = rf.predict(X_test_em)

    rmse = root_mean_squared_error(y_test, y_pred_multi_rf)

    end_time = time.time()

    elapsed_time = end_time - start_time

    n_estimator_time_track_df = n_estimator_time_track_df.append(
        {"n_estimators": n, 'time': elapsed_time, 'RMSE': rmse }, ignore_index=True)

    print(f"Random forest regressor with additional predictors: {rmse:.2f} estimator: {n}")

In [ ]:
# Plot the AIC against Lags

plt.figure(figsize=(14, 4))
plt.plot(n_estimator_time_track_df["time"] ,
         n_estimator_time_track_df["RMSE"],
         label="RMSE across time", color="black")

plt.axhline(y=48.9, color=(1.0, 0.5, 0.5), linestyle='--', label='Threshold')

# Add titles and labels
plt.xlabel('Time (s)', fontsize=14, labelpad=16)
plt.ylabel('RMSE Value', fontsize=14, labelpad=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend()

# plt.savefig("Plots/n_estimator_track_1.png", dpi=400, bbox_inches='tight')

# Show the plot
plt.show()

In [ ]:
# Plot the AIC against Lags

plt.figure(figsize=(14, 4))
#plt.plot(time_track_df["time"] , time_track_df["RMSE"], color="Black")
plt.plot(n_estimator_time_track_df["n_estimators"] 
         , n_estimator_time_track_df["RMSE"], 
         label="RMSE across n_estimators", 
         color="darkblue")

plt.axvline(x=25, color=(1.0, 0.5, 0.5), linestyle='--', label='Threshold')

# Add titles and labels
plt.xlabel('Number of estimators', fontsize=14, labelpad=16)
plt.ylabel('RMSE Value', fontsize=14, labelpad=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend()

# plt.savefig("Plots/n_estimator_track_2.png", dpi=400, bbox_inches='tight')

# Show the plot
plt.show()

In [ ]:
rf = RandomForestRegressor(n_estimators = 25, random_state=42)

rf.fit(X_train_em, y_train)

y_pred_multi_rf = rf.predict(X_test_em)

rmse = root_mean_squared_error(y_test, y_pred_multi_rf)

print(f"Random forest regressor with additional predictors: {rmse:.2f}")

final_df["y_pred_rf_multi"] = y_pred_multi_rf

In [ ]:
tree_reg = DecisionTreeRegressor(random_state=42)

tree_reg.fit(X_train_em.iloc[:,[1]], y_train)

y_pred_dt_uni = tree_reg.predict(X_test_em.iloc[:,[1]])

rmse = root_mean_squared_error(y_test, y_pred_dt_uni)

print(f"Simple decision tree regressor with lagged returns only: {rmse:.2f}")

final_df["y_pred_dt_uni"] = y_pred_dt_uni

In [ ]:
rf = RandomForestRegressor(n_estimators = 25, random_state=42)

rf.fit(X_train_em.iloc[:,[1]], y_train)

y_pred_rf_uni = rf.predict(X_test_em.iloc[:,[1]])

rmse = root_mean_squared_error(y_test, y_pred_rf_uni)

print(f"Simple random forest regressor with lagged returns only: {rmse:.2f}")

final_df["y_pred_rf_uni"] = y_pred_rf_uni

In [ ]:
# train_end_idx and last_pred_idx are parameters that help run the code over multiple executions since running
# it one time for all the test set may take a lot of time.

forecasts_array_dt = rolling_window(main_df, app="dt")
y_pred_multi_dt_rolling = list()
for i in range(0,len(forecasts_array_dt)):
    y_pred_multi_dt_rolling.append(forecasts_array_ar[i].values[0])
    

final_df["y_pred_dt_multi_rolling"] = y_pred_multi_dt_rolling

In [ ]:
# Forecasting the decision tree with the lagged return.

forecasts_array_dt_rolling_uni = rolling_window(main_df[["return", "return_lag_1", "Price"]],
                                    app="dt")
y_pred_dt_rolling_uni = list()
for i in range(0,len(forecasts_array_dt_rolling_uni)):
    y_pred_dt_rolling_uni.append(forecasts_array_dt_rolling_uni[i][0])
    

final_df["y_pred_dt_uni_rolling"] = y_pred_dt_rolling_uni

In [ ]:
# Forecasting the random forest with univariate lagged return.

forecasts_array_rf_rolling_uni = rolling_window(main_df[["return", "return_lag_1", "Price"]],
                                    app="rf")
y_pred_rf_rolling_uni = list()
for i in range(0,len(forecasts_array_rf_rolling_uni)):
    y_pred_rf_rolling_uni.append(forecasts_array_rf_rolling_uni[i][0])
    
# pd.DataFrame(y_pred_rf_rolling_uni).to_csv("y_pred_rf_rolling_uni - Part 1.csv", index=True)

final_df["y_pred_rf_uni_rolling"] = y_pred_rf_rolling_uni

In [ ]:
# Forecasting the random forest with mutiple variables.

forecasts_array_rf_rolling_multi = rolling_window(main_df,
                                    app="rf")
y_pred_rf_multi_rolling = list()
for i in range(0,len(forecasts_array_rf_rolling_multi)):
    y_pred_rf_multi_rolling.append(forecasts_array_rf_rolling_multi[i][0])
    

final_df["y_pred_rf_multi_rolling"] = y_pred_rf_multi_rolling

In [ ]:
# Print all errors of the models.

err = {}

for col in final_df.columns:
    err[col] = root_mean_squared_error(y_test, final_df[[col]])
    
err

In [ ]:
# Modifying column names.

final_df.columns = final_df.columns.str.replace('y_pred_', '')

In [ ]:
# Loading DM tests on all forecasts.

dm_results = {}
p_value_results = {}

for i in tqdm(range(len(final_df.columns)), desc="Loading DM tests"):
    for j in range(i + 1, len(final_df.columns)):
        col1 = final_df.columns[i]
        col2 = final_df.columns[j]
        pred1_lst = final_df[col1].tolist()
        pred2_lst = final_df[col2].tolist()
        
        # Call the dm_test function
        dm_stat, p_value = dm_test(actual_lst=y_test,
                         pred1_lst= pred1_lst, 
                         pred2_lst= pred2_lst)
        
        # Store the result
        dm_results[(col1, col2)] = dm_stat
        p_value_results[(col1, col2)] = p_value


In [ ]:
# Displaying them in a heatmap.
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.lines import Line2D



heatmap_p_value_df = pd.DataFrame(np.zeros((len(final_df.columns), 
                                            len(final_df.columns))), 
                                  index=final_df.columns, 
                                  columns=final_df.columns)

for key, value in p_value_results.items():
    heatmap_p_value_df.loc[key[0], key[1]] = value
    heatmap_p_value_df.loc[key[1], key[0]] = value  # Ensure the matrix is symmetric
    
heatmap_p_value_df = heatmap_p_value_df.applymap(lambda x: 1 if x > 0.05 else 0)
    
mask = np.triu(np.ones_like(heatmap_p_value_df, dtype=bool))



plt.figure(figsize=(12, 8))

cmap = ListedColormap(['lightblue', 'red'])
norm = BoundaryNorm([0, 0.5, 1], cmap.N)

sns.heatmap(heatmap_p_value_df, mask=mask, cmap=cmap, norm=norm, annot=False, 
            linewidths=0.5, linecolor='grey', cbar=False)

legend_elements = [
    Line2D([0], [0], color='lightblue', lw=4, label='<= 0.05'),
    Line2D([0], [0], color='red', lw=4, label='> 0.05')
]

plt.legend(handles=legend_elements, title='P-Value', loc='upper right', fontsize=13, title_fontsize='13')


plt.xticks(rotation=45, ha='right', fontsize=15)
plt.yticks(fontsize=15)

# plt.savefig("Plots/heatmap_p.png", dpi=400, bbox_inches='tight')


plt.show()

In [ ]:
# Displaying them in a heatmap.

heatmap_dm_stat_df = pd.DataFrame(np.zeros((len(final_df.columns), 
                                            len(final_df.columns))), 
                                  index=final_df.columns, 
                                  columns=final_df.columns)

for key, value in dm_results.items():
    heatmap_dm_stat_df.loc[key[0], key[1]] = value
    heatmap_dm_stat_df.loc[key[1], key[0]] = value  # Ensure the matrix is symmetric
    
mask = np.triu(np.ones_like(heatmap_dm_stat_df, dtype=bool))

plt.figure(figsize=(12, 8))
ax = sns.heatmap(heatmap_dm_stat_df, annot=False, mask=mask, cmap='coolwarm', cbar_kws={"shrink": .8})

plt.xticks(rotation=45, ha='right', fontsize=15)
plt.yticks(fontsize=15)

cbar = ax.collections[0].colorbar
cbar.set_label('DM Statistic', rotation=270, labelpad=20, fontsize=15)

# plt.savefig("Plots/heatmap_dm.png", dpi=400, bbox_inches='tight')
plt.show()
